In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import torch

In [2]:
df = pd.read_csv('augmented_train.csv')
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [3]:
df.shape

(12120, 6)

In [4]:
df['language'].value_counts()

English       6870
Chinese        411
Arabic         401
French         390
Swahili        385
Urdu           381
Vietnamese     379
Russian        376
Hindi          374
Greek          372
Thai           371
Spanish        366
Turkish        351
German         351
Bulgarian      342
Name: language, dtype: int64

In [7]:
df['premise'] = df['premise'].str.replace('.', '',regex=True)
df['premise'] = df['premise'].str.replace('http\S+|www.\S+', '',regex=True)

df['hypothesis'] = df['hypothesis'].str.replace('.', '',regex=True)
df['hypothesis'] = df['hypothesis'].str.replace('http\S+|www.\S+', '',regex=True)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

shuffled_df = shuffle(df)

train, test = train_test_split(
                    shuffled_df, train_size = 0.8, test_size = 0.2, shuffle=True, 
                    stratify = shuffled_df['label'])

# x_train1, x_train2, y_train1, y_train2 = train_test_split(
#                     x_train, y_train, 
#                     train_size = 0.5, test_size = 0.5)

In [13]:
x_train = train.drop(['label', 'id'], axis = 1)
y_train = train['label']

x_test = test.drop(['label', 'id'], axis = 1)
y_test = test['label']

In [15]:
del df, shuffled_df

In [14]:
x_train.shape

(9696, 4)

In [15]:
y_train.shape

(9696,)

In [24]:
import torch
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
from tqdm import tqdm
def extract_mbert_cls_only(list_of_premises=[], list_of_hypothesis=[], batch_size=16):
    cls_hidden_states = []
    if len(list_of_premises)<batch_size: #go longer ones we should batch
        tknzed= tokenizer(list_of_premises, list_of_hypothesis, return_tensors="pt", padding=True)
        b=model(**tknzed)
        cls_hidden_states = torch.squeeze(b.last_hidden_state[:,0,:])
    else:
        for i in tqdm(range(0,len(list_of_premises),batch_size)):    
            tknzed= tokenizer(list_of_premises[i:min(len(list_of_premises),i+batch_size)], list_of_hypothesis[i:min(len(list_of_hypothesis),i+batch_size)], return_tensors="pt", padding=True)
            b=model(**tknzed)
            cls_hidden_states.append(torch.squeeze(b.last_hidden_state[:,0,:]))
#             print(b.last_hidden_state[:,0,:].shape)
#             print(b.last_hidden_state[:,1:,:].shape)
    return cls_hidden_states

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### For only CLS features

In [25]:
for i in range(0, len(x_train), 400):
    reduced_x_train = x_train[i: i + 400]
    cls_reduced_x_train = extract_mbert_cls_only(list(reduced_x_train['premise']), list(reduced_x_train['hypothesis']))
    cls_reduced_x_train_torch = torch.cat(cls_reduced_x_train)
    cls_reduced_x_train_features = cls_reduced_x_train_torch.cpu().detach().numpy()
    print(i)
    if i != 0:
        cls_x_train_features = np.concatenate((cls_x_train_features, cls_reduced_x_train_features), axis = 0)
    else:
        cls_x_train_features = cls_reduced_x_train_features
#     reduced_y_train = np.asarray(reduced_y_train)
    print(cls_x_train_features.shape)
    del reduced_x_train, cls_reduced_x_train, cls_reduced_x_train_torch, cls_reduced_x_train_features




  0%|                                                    | 0/25 [00:00<?, ?it/s]


  4%|█▊                                          | 1/25 [00:00<00:09,  2.46it/s]


  8%|███▌                                        | 2/25 [00:01<00:14,  1.59it/s]


 12%|█████▎                                      | 3/25 [00:01<00:12,  1.73it/s]


 16%|███████                                     | 4/25 [00:02<00:11,  1.78it/s]


 20%|████████▊                                   | 5/25 [00:02<00:11,  1.71it/s]


 24%|██████████▌                                 | 6/25 [00:03<00:09,  1.92it/s]


 28%|████████████▎                               | 7/25 [00:03<00:09,  1.83it/s]


 32%|██████████████                              | 8/25 [00:04<00:08,  1.94it/s]


 36%|███████████████▊                            | 9/25 [00:04<00:08,  1.80it/s]


 40%|█████████████████▏                         | 10/25 [00:05<00:09,  1.59it/s]


 44%|██████████████████▉                        | 11/25 [00:06<00:09,  1.54it/s]


 

0
(400, 768)





  0%|                                                    | 0/25 [00:00<?, ?it/s]


  4%|█▊                                          | 1/25 [00:00<00:15,  1.57it/s]


  8%|███▌                                        | 2/25 [00:01<00:12,  1.78it/s]


 12%|█████▎                                      | 3/25 [00:01<00:12,  1.78it/s]


 16%|███████                                     | 4/25 [00:02<00:10,  1.96it/s]


 20%|████████▊                                   | 5/25 [00:02<00:08,  2.22it/s]


 24%|██████████▌                                 | 6/25 [00:02<00:08,  2.22it/s]


 28%|████████████▎                               | 7/25 [00:03<00:07,  2.37it/s]


 32%|██████████████                              | 8/25 [00:03<00:07,  2.38it/s]


 36%|███████████████▊                            | 9/25 [00:04<00:10,  1.55it/s]


 40%|█████████████████▏                         | 10/25 [00:05<00:11,  1.35it/s]


 44%|██████████████████▉                        | 11/25 [00:06<00:10,  1.35it/s]


 

400
(800, 768)





  0%|                                                    | 0/25 [00:00<?, ?it/s]


  4%|█▊                                          | 1/25 [00:00<00:23,  1.01it/s]


  8%|███▌                                        | 2/25 [00:01<00:16,  1.37it/s]


 12%|█████▎                                      | 3/25 [00:02<00:13,  1.65it/s]


 16%|███████                                     | 4/25 [00:02<00:11,  1.89it/s]


 20%|████████▊                                   | 5/25 [00:02<00:10,  1.89it/s]


 24%|██████████▌                                 | 6/25 [00:03<00:09,  1.99it/s]


 28%|████████████▎                               | 7/25 [00:03<00:09,  1.92it/s]


 32%|██████████████                              | 8/25 [00:04<00:08,  1.97it/s]


 36%|███████████████▊                            | 9/25 [00:04<00:08,  1.95it/s]


 40%|█████████████████▏                         | 10/25 [00:05<00:08,  1.75it/s]


 44%|██████████████████▉                        | 11/25 [00:07<00:15,  1.08s/it]


 

...

8800
(9200, 768)


100%|███████████████████████████████████████████| 25/25 [00:16<00:00,  1.55it/s]


9200
(9600, 768)


100%|█████████████████████████████████████████████| 6/6 [00:02<00:00,  2.10it/s]

9600
(9696, 768)


In [29]:
with open('train_cls_features_mbert.npy', 'wb') as f:
    np.save(f, cls_x_train_features)
    
with open('train_labels_mbert.npy', 'wb') as f:
    np.save(f, y_train)

In [27]:
for i in range(0, len(x_test), 400):
    reduced_x_test = x_test[i: i + 400]
    cls_reduced_x_test = extract_mbert_cls_only(list(reduced_x_test['premise']), list(reduced_x_test['hypothesis']))
    cls_reduced_x_test_torch = torch.cat(cls_reduced_x_test)
    cls_reduced_x_test_features = cls_reduced_x_test_torch.cpu().detach().numpy()
    print(i)
    if i != 0:
        cls_x_test_features = np.concatenate((cls_x_test_features, cls_reduced_x_test_features), axis = 0)
    else:
        cls_x_test_features = cls_reduced_x_test_features
#     reduced_y_train = np.asarray(reduced_y_train)
    print(cls_x_test_features.shape)
    del reduced_x_test, cls_reduced_x_test, cls_reduced_x_test_torch, cls_reduced_x_test_features

100%|███████████████████████████████████████████| 25/25 [00:16<00:00,  1.54it/s]


0
(400, 768)


100%|███████████████████████████████████████████| 25/25 [00:17<00:00,  1.40it/s]


400
(800, 768)


100%|███████████████████████████████████████████| 25/25 [00:18<00:00,  1.35it/s]


...

1600
(2000, 768)


100%|███████████████████████████████████████████| 25/25 [00:15<00:00,  1.57it/s]


2000
(2400, 768)


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  2.92it/s]

2400
(2424, 768)


In [30]:
with open('test_cls_features_mbert.npy', 'wb') as f:
    np.save(f, cls_x_test_features)
    
with open('test_labels_mbert.npy', 'wb') as f:
    np.save(f, y_test)

## Loading features

In [3]:
cls_x_train = np.load('train_cls_features_mbert.npy', allow_pickle=True)
y_train = np.load('train_labels_mbert.npy', allow_pickle=True)

In [4]:
cls_x_train.shape

(9696, 768)

In [5]:
y_train.shape

(9696,)

In [6]:
cls_x_test = np.load('test_cls_features_mbert.npy', allow_pickle=True)
y_test = np.load('test_labels_mbert.npy', allow_pickle=True)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(cls_x_train)
simple_x_train = scaler.transform(cls_x_train)

simple_x_test =scaler.transform(cls_x_test)

In [8]:
simple_x_train.shape

(9696, 768)

In [9]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver = "liblinear")
clf.fit(simple_x_train, y_train)
pred = clf.predict(simple_x_test)
pred = classification_report(y_test, pred)
print(pred)

              precision    recall  f1-score   support

           0       0.55      0.57      0.56       835
           1       0.54      0.50      0.52       776
           2       0.51      0.52      0.51       813

    accuracy                           0.53      2424
   macro avg       0.53      0.53      0.53      2424
weighted avg       0.53      0.53      0.53      2424



In [10]:
model = LinearSVC(max_iter = 10000)

model.fit(simple_x_train, y_train)
pred = model.predict(simple_x_test)
pred = classification_report(y_test, pred)
print(pred)

/Users/geetshingi/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.54      0.57      0.55       835
           1       0.54      0.50      0.52       776
           2       0.51      0.52      0.52       813

    accuracy                           0.53      2424
   macro avg       0.53      0.53      0.53      2424
weighted avg       0.53      0.53      0.53      2424



In [57]:
model = Perceptron()

model.fit(simple_x_train, y_train)
pred = model.predict(simple_x_test)
pred = classification_report(y_test, pred)
print(pred)

              precision    recall  f1-score   support

           0       0.51      0.51      0.51       835
           1       0.47      0.54      0.50       776
           2       0.47      0.41      0.44       813

    accuracy                           0.48      2424
   macro avg       0.48      0.48      0.48      2424
weighted avg       0.48      0.48      0.48      2424



In [59]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(simple_x_train, y_train)
pred = model.predict(simple_x_test)
pred = classification_report(y_test, pred)
print(pred)

              precision    recall  f1-score   support

           0       0.50      0.53      0.52       835
           1       0.44      0.41      0.43       776
           2       0.45      0.46      0.46       813

    accuracy                           0.47      2424
   macro avg       0.47      0.47      0.47      2424
weighted avg       0.47      0.47      0.47      2424



In [21]:
x_train_torch = torch.from_numpy(cls_x_train)
x_test_torch = torch.from_numpy(cls_x_test)

In [22]:
zipper = lambda x,y : list(zip(x,list(y)))

In [23]:
train_torch = zipper(x_train_torch, y_train)
test_torch = zipper(x_test_torch, y_test)

train_loader = torch.utils.data.DataLoader(train_torch, batch_size=256)
test_loader = torch.utils.data.DataLoader(test_torch, batch_size=256)

In [24]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): 
    
    def __init__(self):
        super(Net, self).__init__() 
#         self.gru = nn.GRU(input_size=768, hidden_size=32, batch_first=True) 
        self.fc1 = nn.Linear(768,3)
#         self.fc2 = nn.Linear(256, 64)
#         self.fc3 = nn.Linear(128, 3)
        self.relu = nn.ReLU()

#         self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
#         out, hn = self.gru(x, torch.randn(1, len(x), 32))
#         out = self.fc1(self.relu(x))
#         out = self.fc2(self.relu(out))
        out = self.fc1(x)
        return out

In [53]:
linear_model = Net()
print(linear_model)

Net(
  (fc1): Linear(in_features=768, out_features=3, bias=True)
  (relu): ReLU()
)


In [54]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(linear_model.parameters(), lr=0.03)

In [55]:
# number of epochs to train the model
n_epochs = 1500

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    linear_model.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = linear_model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    linear_model.eval() # prep model for evaluation
    for data, target in test_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = linear_model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(test_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}' \
          .format(epoch+1, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}). \
        Saving model ...'.format(valid_loss_min, valid_loss))
        
        torch.save(linear_model.state_dict(), 'mbert_model_linear.pt')
        valid_loss_min = valid_loss
        

Epoch: 1 	Training Loss: 1.113914 	Validation Loss: 1.109428
Validation loss decreased (inf --> 1.109428).         Saving model ...
Epoch: 2 	Training Loss: 1.108770 	Validation Loss: 1.103706
Validation loss decreased (1.109428 --> 1.103706).         Saving model ...
Epoch: 3 	Training Loss: 1.101950 	Validation Loss: 1.098257
Validation loss decreased (1.103706 --> 1.098257).         Saving model ...
Epoch: 4 	Training Loss: 1.095780 	Validation Loss: 1.093183
Validation loss decreased (1.098257 --> 1.093183).         Saving model ...
Epoch: 5 	Training Loss: 1.090151 	Validation Loss: 1.088501
Validation loss decreased (1.093183 --> 1.088501).         Saving model ...
Epoch: 6 	Training Loss: 1.084995 	Validation Loss: 1.084190
Validation loss decreased (1.088501 --> 1.084190).         Saving model ...
Epoch: 7 	Training Loss: 1.080256 	Validation Loss: 1.080219
Validation loss decreased (1.084190 --> 1.080219).         Saving model ...
Epoch: 8 	Training Loss: 1.075885 	Validation 

Epoch: 67 	Training Loss: 0.991695 	Validation Loss: 1.008709
Validation loss decreased (1.009175 --> 1.008709).         Saving model ...
Epoch: 68 	Training Loss: 0.991075 	Validation Loss: 1.008250
Validation loss decreased (1.008709 --> 1.008250).         Saving model ...
Epoch: 69 	Training Loss: 0.990463 	Validation Loss: 1.007798
Validation loss decreased (1.008250 --> 1.007798).         Saving model ...
Epoch: 70 	Training Loss: 0.989860 	Validation Loss: 1.007354
Validation loss decreased (1.007798 --> 1.007354).         Saving model ...
Epoch: 71 	Training Loss: 0.989264 	Validation Loss: 1.006917
Validation loss decreased (1.007354 --> 1.006917).         Saving model ...
Epoch: 72 	Training Loss: 0.988677 	Validation Loss: 1.006486
Validation loss decreased (1.006917 --> 1.006486).         Saving model ...
Epoch: 73 	Training Loss: 0.988098 	Validation Loss: 1.006062
Validation loss decreased (1.006486 --> 1.006062).         Saving model ...
Epoch: 74 	Training Loss: 0.987526

Epoch: 131 	Training Loss: 0.963514 	Validation Loss: 0.988978
Validation loss decreased (0.989186 --> 0.988978).         Saving model ...
Epoch: 132 	Training Loss: 0.963198 	Validation Loss: 0.988772
Validation loss decreased (0.988978 --> 0.988772).         Saving model ...
Epoch: 133 	Training Loss: 0.962883 	Validation Loss: 0.988569
Validation loss decreased (0.988772 --> 0.988569).         Saving model ...
Epoch: 134 	Training Loss: 0.962572 	Validation Loss: 0.988367
Validation loss decreased (0.988569 --> 0.988367).         Saving model ...
Epoch: 135 	Training Loss: 0.962263 	Validation Loss: 0.988167
Validation loss decreased (0.988367 --> 0.988167).         Saving model ...
Epoch: 136 	Training Loss: 0.961956 	Validation Loss: 0.987969
Validation loss decreased (0.988167 --> 0.987969).         Saving model ...
Epoch: 137 	Training Loss: 0.961652 	Validation Loss: 0.987774
Validation loss decreased (0.987969 --> 0.987774).         Saving model ...
Epoch: 138 	Training Loss: 

Epoch: 192 	Training Loss: 0.947856 	Validation Loss: 0.979380
Validation loss decreased (0.979499 --> 0.979380).         Saving model ...
Epoch: 193 	Training Loss: 0.947648 	Validation Loss: 0.979261
Validation loss decreased (0.979380 --> 0.979261).         Saving model ...
Epoch: 194 	Training Loss: 0.947441 	Validation Loss: 0.979143
Validation loss decreased (0.979261 --> 0.979143).         Saving model ...
Epoch: 195 	Training Loss: 0.947235 	Validation Loss: 0.979026
Validation loss decreased (0.979143 --> 0.979026).         Saving model ...
Epoch: 196 	Training Loss: 0.947030 	Validation Loss: 0.978910
Validation loss decreased (0.979026 --> 0.978910).         Saving model ...
Epoch: 197 	Training Loss: 0.946826 	Validation Loss: 0.978795
Validation loss decreased (0.978910 --> 0.978795).         Saving model ...
Epoch: 198 	Training Loss: 0.946624 	Validation Loss: 0.978681
Validation loss decreased (0.978795 --> 0.978681).         Saving model ...
Epoch: 199 	Training Loss: 

Epoch: 261 	Training Loss: 0.935778 	Validation Loss: 0.972948
Validation loss decreased (0.973021 --> 0.972948).         Saving model ...
Epoch: 262 	Training Loss: 0.935630 	Validation Loss: 0.972876
Validation loss decreased (0.972948 --> 0.972876).         Saving model ...
Epoch: 263 	Training Loss: 0.935484 	Validation Loss: 0.972804
Validation loss decreased (0.972876 --> 0.972804).         Saving model ...
Epoch: 264 	Training Loss: 0.935338 	Validation Loss: 0.972733
Validation loss decreased (0.972804 --> 0.972733).         Saving model ...
Epoch: 265 	Training Loss: 0.935192 	Validation Loss: 0.972662
Validation loss decreased (0.972733 --> 0.972662).         Saving model ...
Epoch: 266 	Training Loss: 0.935047 	Validation Loss: 0.972591
Validation loss decreased (0.972662 --> 0.972591).         Saving model ...
Epoch: 267 	Training Loss: 0.934903 	Validation Loss: 0.972521
Validation loss decreased (0.972591 --> 0.972521).         Saving model ...
Epoch: 268 	Training Loss: 

...

Epoch: 1403 	Training Loss: 0.877242 	Validation Loss: 0.957750
Epoch: 1404 	Training Loss: 0.877220 	Validation Loss: 0.957751
Epoch: 1405 	Training Loss: 0.877197 	Validation Loss: 0.957751
Epoch: 1406 	Training Loss: 0.877175 	Validation Loss: 0.957751
Epoch: 1407 	Training Loss: 0.877153 	Validation Loss: 0.957752
Epoch: 1408 	Training Loss: 0.877131 	Validation Loss: 0.957752
Epoch: 1409 	Training Loss: 0.877108 	Validation Loss: 0.957753
Epoch: 1410 	Training Loss: 0.877086 	Validation Loss: 0.957753
Epoch: 1411 	Training Loss: 0.877064 	Validation Loss: 0.957754
Epoch: 1412 	Training Loss: 0.877042 	Validation Loss: 0.957755
Epoch: 1413 	Training Loss: 0.877020 	Validation Loss: 0.957755
Epoch: 1414 	Training Loss: 0.876998 	Validation Loss: 0.957756
Epoch: 1415 	Training Loss: 0.876976 	Validation Loss: 0.957756
Epoch: 1416 	Training Loss: 0.876954 	Validation Loss: 0.957757
Epoch: 1417 	Training Loss: 0.876932 	Validation Loss: 0.957757
Epoch: 1418 	Training Loss: 0.876910 	Va

Epoch: 1540 	Training Loss: 0.874350 	Validation Loss: 0.957870
Epoch: 1541 	Training Loss: 0.874330 	Validation Loss: 0.957871
Epoch: 1542 	Training Loss: 0.874310 	Validation Loss: 0.957873
Epoch: 1543 	Training Loss: 0.874290 	Validation Loss: 0.957874
Epoch: 1544 	Training Loss: 0.874270 	Validation Loss: 0.957875
Epoch: 1545 	Training Loss: 0.874250 	Validation Loss: 0.957876
Epoch: 1546 	Training Loss: 0.874230 	Validation Loss: 0.957878
Epoch: 1547 	Training Loss: 0.874210 	Validation Loss: 0.957879
Epoch: 1548 	Training Loss: 0.874190 	Validation Loss: 0.957880
Epoch: 1549 	Training Loss: 0.874170 	Validation Loss: 0.957882
Epoch: 1550 	Training Loss: 0.874150 	Validation Loss: 0.957883
Epoch: 1551 	Training Loss: 0.874131 	Validation Loss: 0.957884
Epoch: 1552 	Training Loss: 0.874111 	Validation Loss: 0.957886
Epoch: 1553 	Training Loss: 0.874091 	Validation Loss: 0.957887
Epoch: 1554 	Training Loss: 0.874071 	Validation Loss: 0.957888
Epoch: 1555 	Training Loss: 0.874051 	Va

Epoch: 1671 	Training Loss: 0.871854 	Validation Loss: 0.958075
Epoch: 1672 	Training Loss: 0.871836 	Validation Loss: 0.958076
Epoch: 1673 	Training Loss: 0.871818 	Validation Loss: 0.958078
Epoch: 1674 	Training Loss: 0.871800 	Validation Loss: 0.958080
Epoch: 1675 	Training Loss: 0.871782 	Validation Loss: 0.958082
Epoch: 1676 	Training Loss: 0.871763 	Validation Loss: 0.958084
Epoch: 1677 	Training Loss: 0.871745 	Validation Loss: 0.958086
Epoch: 1678 	Training Loss: 0.871727 	Validation Loss: 0.958088
Epoch: 1679 	Training Loss: 0.871709 	Validation Loss: 0.958089
Epoch: 1680 	Training Loss: 0.871691 	Validation Loss: 0.958091
Epoch: 1681 	Training Loss: 0.871673 	Validation Loss: 0.958093
Epoch: 1682 	Training Loss: 0.871655 	Validation Loss: 0.958095
Epoch: 1683 	Training Loss: 0.871637 	Validation Loss: 0.958097
Epoch: 1684 	Training Loss: 0.871619 	Validation Loss: 0.958099
Epoch: 1685 	Training Loss: 0.871601 	Validation Loss: 0.958101
Epoch: 1686 	Training Loss: 0.871583 	Va

Epoch: 1802 	Training Loss: 0.869581 	Validation Loss: 0.958348
Epoch: 1803 	Training Loss: 0.869565 	Validation Loss: 0.958351
Epoch: 1804 	Training Loss: 0.869548 	Validation Loss: 0.958353
Epoch: 1805 	Training Loss: 0.869532 	Validation Loss: 0.958355
Epoch: 1806 	Training Loss: 0.869515 	Validation Loss: 0.958358
Epoch: 1807 	Training Loss: 0.869498 	Validation Loss: 0.958360
Epoch: 1808 	Training Loss: 0.869482 	Validation Loss: 0.958362
Epoch: 1809 	Training Loss: 0.869465 	Validation Loss: 0.958365
Epoch: 1810 	Training Loss: 0.869449 	Validation Loss: 0.958367
Epoch: 1811 	Training Loss: 0.869432 	Validation Loss: 0.958369
Epoch: 1812 	Training Loss: 0.869416 	Validation Loss: 0.958372
Epoch: 1813 	Training Loss: 0.869399 	Validation Loss: 0.958374
Epoch: 1814 	Training Loss: 0.869383 	Validation Loss: 0.958376
Epoch: 1815 	Training Loss: 0.869367 	Validation Loss: 0.958379
Epoch: 1816 	Training Loss: 0.869350 	Validation Loss: 0.958381
Epoch: 1817 	Training Loss: 0.869334 	Va

Epoch: 1935 	Training Loss: 0.867469 	Validation Loss: 0.958683
Epoch: 1936 	Training Loss: 0.867454 	Validation Loss: 0.958686
Epoch: 1937 	Training Loss: 0.867438 	Validation Loss: 0.958688
Epoch: 1938 	Training Loss: 0.867423 	Validation Loss: 0.958691
Epoch: 1939 	Training Loss: 0.867408 	Validation Loss: 0.958694
Epoch: 1940 	Training Loss: 0.867393 	Validation Loss: 0.958697
Epoch: 1941 	Training Loss: 0.867378 	Validation Loss: 0.958699
Epoch: 1942 	Training Loss: 0.867363 	Validation Loss: 0.958702
Epoch: 1943 	Training Loss: 0.867347 	Validation Loss: 0.958705
Epoch: 1944 	Training Loss: 0.867332 	Validation Loss: 0.958707
Epoch: 1945 	Training Loss: 0.867317 	Validation Loss: 0.958710
Epoch: 1946 	Training Loss: 0.867302 	Validation Loss: 0.958713
Epoch: 1947 	Training Loss: 0.867287 	Validation Loss: 0.958716
Epoch: 1948 	Training Loss: 0.867272 	Validation Loss: 0.958718
Epoch: 1949 	Training Loss: 0.867257 	Validation Loss: 0.958721
Epoch: 1950 	Training Loss: 0.867242 	Va

In [56]:
linear_model = Net()
linear_model.load_state_dict(torch.load("mbert_model_linear.pt"))
linear_model.eval()

pred_list = torch.zeros(0, dtype=torch.long)
target_list = torch.zeros(0, dtype=torch.long)
train_count = 0
count = 0

train_pred_list = torch.zeros(0, dtype=torch.long)
train_target_list = torch.zeros(0, dtype=torch.long)

for data, target in train_loader:
    output = linear_model(data)
    _, preds = torch.max(output, 1)
    train_pred_list = torch.cat([train_pred_list, preds.view(-1)])
    train_target_list = torch.cat([train_target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             train_count+=1
            
# print("Training Accuracy =", train_count / len(train_loader.dataset))
train_result = classification_report(train_pred_list.numpy(), 
                                     train_target_list.numpy())
print("Training Classification report: \n", train_result)
            
for data, target in test_loader:
    output = linear_model(data)
    _, preds = torch.max(output, 1) 
    pred_list = torch.cat([pred_list, preds.view(-1)])
    target_list = torch.cat([target_list, target.view(-1)])
#     for x in range(16):
#         if preds[x] == target[x]: 
#             count+=1

# print("Testing Accuracy =", count / len(test_loader.dataset))

result = classification_report(pred_list.numpy(), target_list.numpy())
print("Testing Classification report: \n", result)

Training Classification report: 
               precision    recall  f1-score   support

           0       0.66      0.60      0.63      3679
           1       0.55      0.61      0.58      2809
           2       0.58      0.59      0.58      3208

    accuracy                           0.60      9696
   macro avg       0.60      0.60      0.60      9696
weighted avg       0.60      0.60      0.60      9696

Testing Classification report: 
               precision    recall  f1-score   support

           0       0.60      0.54      0.57       922
           1       0.49      0.55      0.52       702
           2       0.52      0.53      0.52       800

    accuracy                           0.54      2424
   macro avg       0.54      0.54      0.54      2424
weighted avg       0.54      0.54      0.54      2424

